
<h1><center>Многоязычная классификация текстов на основе BERT</center></h1>

Мы продолжим знакомство с моделями на основе архитектуры Трансформер. 
На этом занятии мы рассмотрим классификатор на основе модели RoBERTa (модель на базе BERT), дообученный для решения задачи Natural Language Inference (NLI). В этой задаче нам требуется предсказать, если ли логическая связь между двумя предложениями: следует ли одного из другого.  Этот классификатор, предобученный на задаче NLI, позволит нам решать задачу классификации в Zero-shot постановке. В такой постановке модели НЕ требуется обучение на размеченных данных, поскольку любую задачу мы можем свести к задаче NLI –- мы рассмотрим, как именно в примерах ниже.

Мы загрузим заранее обученный классификатор и будем использовать его, чтообы классифицировать тексты на нужные нам классы. 

Два важных отличия рассматриваемой сегодня модели от тех, с которыми мы работали на предыдущих занятиях:
- Подход zero-shot не требует обучения модели, нам следует лишь подать классификатору текст и названия нужных нам классов. Модель будет строить предсказание на основе своего внутреннего представления, обученного заранее.
- Данная модель многоязычна. Мы будем работать с текстами, написанными на разных языках.

## Классификация текстов на английском языке

Установим библиотеку transformers и загрузим модуль pipeline, в котором находится нужная нам модель.

In [2]:
!pip install transformers==3.1.0

  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893262 sha256=3c23f73df7e10b1aa7bc1f5c7757a4e83e5cf6babb52e94ea11edbfeb162b373
  Stored in directory: c:\users\basilio\appdata\local\pip\cache\wheels\7b\78\f4\27d43a65043e1b75dbddaa421b573eddc67e712be4b1c80677
Successfully built sacremoses


In [1]:
from transformers import pipeline

In [2]:
classifier = pipeline("zero-shot-classification")
# classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
import pandas as pd
import regex as re
from tqdm.notebook import tqdm
import os

site = 'http://www.lada-vesta.net/'
folder_names = [
    ['0', 'Отзывы', 'f=25', 'LADA VESTA'],
    ['1', 'Двигатель', 'f=14', 'LADA VESTA'],
    ['2', 'Трансмиссия', 'f=15', 'LADA VESTA'],
    ['3', 'Ходовая', 'f=16', 'LADA VESTA'],
    ['4', 'Колеса', '', 'LADA VESTA'],
    ['5', 'Электрооборудование', 'f=17', 'LADA VESTA'],
    ['6', 'Мультимедиа', '', 'LADA VESTA'],
    ['7', 'Кузов', 'f=18', 'LADA VESTA'],
    ['8', 'Жидкости', 'f=19', 'LADA VESTA'],
    ['9', 'Сервис', 'f=42', 'LADA VESTA'],
    ['10', 'Эксплуатация', 'f=23', 'LADA VESTA'],
    ['11', 'Тюнинг', 'f=24', 'LADA VESTA']
]

In [4]:
folder_name = folder_names[0]
f_name = folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv'
f_name

'LADA VESTA_0_Отзывы.csv'

In [5]:
mess = pd.read_csv(f_name, quotechar="'")
mess

,Code,Folder,Theme,Date,Time,Author,Message
0,0,Отзывы,Пробег наших Вест!!!,07.10.2016,17:37,Роман 72,Максимальный пробег в теме: Задумал создать та...
1,0,Отзывы,Пробег наших Вест!!!,07.10.2016,18:07,Сергей 74,"Комфорт +мультимедиа МТ, 3 тыс. Все норм."
2,0,Отзывы,Пробег наших Вест!!!,07.10.2016,19:29,Владимир34,"Люкс МТ,3500км.Напрягает работа подвески(крип,..."
3,0,Отзывы,Пробег наших Вест!!!,07.10.2016,19:38,timtl,"Люкс XV line, 12000км. Заменили стойку стабили..."
4,0,Отзывы,Пробег наших Вест!!!,07.10.2016,23:03,SAlex,Бомж-комплектация. 17500км. Сопливил движок в ...
...,...,...,...,...,...,...,...
12688,0,Отзывы,"Отзыв про ""Весточку"" XoXeL - Новосибирск",28.12.2015,06:46,Rus24lan,"XoXeL, Привет, что за автосигнализация постави..."
12689,0,Отзывы,Авангард - трейд Белгород,30.11.2015,08:58,Daniil111,Купил в Белгороде Весту на механике в комплект...
12690,0,Отзывы,Авангард - трейд Белгород,30.11.2015,10:52,ПотомуЧтоГладиолус,Поздр! Ждем отзывов
12691,0,Отзывы,Авангард - трейд Белгород,30.11.2015,11:00,Ladavod,Что именно так зацепило на тест драйве в сравн...


In [6]:
def rate_sentiment(message):
    sequences = [message]
    candidate_labels = ["positive", "neutral", "negative"]
    hypothesis_template = "The sentiment of this review is {}."

    result = classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)
    return result['labels'][0]

Решаем задачу классификации по тональности: классифицируем сообщения на негативный, нейтральный и позитивный классы.


Модель, лежащая в основе данного классификатора, была обучена на задаче Natural Language Inference (NLI). Задача состояла в следующем: по двум входящим текстам требовалось определить, является ли один из них продолжением другого.

Такую модель можно использвать в задаче zero-shot классификации, если свести классификацию к задаче NLI. Для этого модели на вход подаются два текста:
- текст, который нужно классифицировать (*предпосылка*)
- текст шаблона, в который вставлено название нужного класса (*гипотеза*)


Если с точки зрения NLI-модели текст гипотезы продолжает текст предпосылки, то мы можем заключить, что классифицируемый текст относится к соответствующему классу.

По умолчанию заданные метки классов вставляются в шаблон `This example is {class_name}.` 

Во многих случаях такой подход работает достаточно хорошо, но в некоторых задачах качество классификации можно улучшить, используя более подходящий шаблон.

Второй пример выше несколько сложнее первого, поэтому модель предсказывает негативный класс менее уверенно. 

Попробуем повысить уверенность классификации, используя более подходящий шаблон для данной задачи. Вместо шаблона по умолчанию `This example is {}.`, мы будем использовать `The sentiment of this review is {}.` (здесь `{}` будет заменено на название класса)

In [23]:
sequences = [
    "Видно в Кемеровской области дороги не Российские или нервы крепкие все равно завидую",
    "Машине 10 дней механика в максимальной комплектации пробег 200 км появилось тарахтение как я думаю в коробке передач так как если сцепление выжимаешь то тишина. Эх...."
]
candidate_labels = ["positive", "neutral", "negative"]
hypothesis_template = "The sentiment of this review is {}."

#classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)
rate_sentiment(sequences[0])

'negative'

In [11]:
sequences = mess[mess['Theme']=='Отзыв от Игоря']
candidate_labels = ["positive", "neutral", "negative"]
hypothesis_template = "The sentiment of this review is {}."

classifier(list(sequences.Message[:5]), candidate_labels, hypothesis_template=hypothesis_template)

[{'sequence': 'Машина мне понравилась,комплектация за 585000 .Прошел 450 км правда уже обнаружил стук в стойке при прохождении на скорость 25 км полицейского.Стук будто не закручено что-то. Еще некий звук из под капота шуршащий подшипник ремня ,что ли . Буду ждать пока посильнее проявятся проблемы потом поеду в сервис. А чуть не забыл еще сигнал тупит отрегулировать надо . Звук будто как на копейке стояли с регулировкой и при этом не настроен -хотел сам добраться до него, но не доберёшься. Не хватает освещения на заднем сиденье. Добавлено через 3 минуты вин у меня последние цифры 0963 .Брал её в ПУШКИНО.',
  'labels': ['negative', 'positive', 'neutral'],
  'scores': [0.5261519551277161, 0.24874958395957947, 0.2250984013080597]},
 {'sequence': 'Рекомендую сразу обратиться к дилеру, такой асимметричности не должно быть на отбое. Результаты - сюда. Попрошу разместить также фото, для нашей клубной коллекции, тут: http://www.lada-vesta.net/showthread.php?t=503',
  'labels': ['negative', 'po


Используя более точные и более подходящие к нашему контексту шаблоны, мы можем получить более точную классификацию.

## Многоязычная классификация

Данный классификатор можно использовать не только для английского языка, но и для ряда других. Для этого была обучена кросс-язычная модель на базе модели XLM RoBERTa.

Чтобы использовать эту модель, достаточно добавить дополнительный параметр ```model``` при объявлении классификатора.

In [2]:
classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')

In [57]:
import fitz
import os
import regex as re
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
mess = pd.read_csv('./Data/collection.txt', quotechar='"')

(60152, 2)

In [52]:
print(mess.iloc[:10,1])

0    Issued: January 2009 Code of Ethics Revised: P...
1    Integrity The integrity of internal auditors e...
2    Objectivity Internal auditors exhibit the high...
3    Internal auditors make a balanced assessment o...
4    Confidentiality Internal auditors respect the ...
5    Competency Internal auditors apply the knowled...
6                                   Rules of Conduct 1
7                       Integrity Internal auditors: 1
8    Shall perform their work with honesty, diligen...
9    Shall observe the law and make disclosures exp...
Name:  Text, dtype: object


In [53]:
question = input()

corporate governance




Модель позволяет использовать любую комбинацию языков. Можно, например, классифицировать русский текст с английскими названиями классов.

In [58]:
probabilities = []
candidate_labels = [question]
hypothesis_template = "This sentence is about {}."
n = len(mess)
for i in tqdm(range(n)):
    sequence = mess.iloc[i, 1]
    result = classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)
    probabilities.append([result['scores'][0], i])

KeyboardInterrupt: 

In [60]:
i = max(probabilities)[1]
print(mess.iloc[i])

Doc                                 Standards rus 2017.pdf
 Text    совет директоров, наблюдательный совет или сов...
Name: 803, dtype: object


In [64]:
probabilities.sort(reverse=True)
probabilities[:10]

[[0.9994304776191711, 803],
 [0.9993367195129395, 744],
 [0.9992428421974182, 740],
 [0.9992086887359619, 1540],
 [0.9991915822029114, 1802],
 [0.9991769790649414, 935],
 [0.9991758465766907, 1637],
 [0.9991623759269714, 1156],
 [0.9991623759269714, 47],
 [0.9991306662559509, 709]]

Теперь можно сделать то же самое, только поменять названия классов на французские.

In [6]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "santé publique", "politique"]

classifier(sequence, candidate_labels)

{'sequence': 'За кого вы голосуете в 2020 году?',
 'labels': ['politique', 'Europe', 'santé publique'],
 'scores': [0.9726155400276184, 0.017128445208072662, 0.010256034322082996]}


Как было отмечено выше, по умолчанию в модели используется шаблон на английском языке, `This text is {}.`
В случае, если мы работаем с другим языком, имеет смысл поменять данный шаблон на аналогичный, написанный на нужном нам языке.

In [7]:
sequence = "¿A quién vas a votar en 2020?"
candidate_labels = ["Europa", "salud pública", "política"]
hypothesis_template = "Este ejemplo es {}."

classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)

{'sequence': '¿A quién vas a votar en 2020?',
 'labels': ['política', 'Europa', 'salud pública'],
 'scores': [0.9109594225883484, 0.0595475435256958, 0.02949300780892372]}

Модель была дообучена на корпусе XNLI, в состав которого входят тексты на 15 языках: английский, арабский, болгарский, вьетнамский, греческий, испанский, немецкий, русский, суахили, тайский, турецкий, урду, французский и хинди. 

Базовая модель RoBERTa, на основе которой была построена данная модель, была обучена на 100 языках, поэтому для каждого из этих языков эта модель будет также работать в некоторой степени.  

## Итоги

Сегодня мы познакомились с моделью на базе RoBERTa для решения задачи NLI.

Мы научились использовать построенный на основе данной модели классификатор для классификации текстов на разных языках по разным классам, названия которых мы задавали сами.

На следующих занятиях мы продолжим изучение моделей, основанных на архитектуре BERT и рассмотрим другие задачи, которые можем решать с их помощью.